In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import os
from keras import layers
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_path = os.path.join('Dataset1', 'Train')

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

In [ ]:
train_ds = train_datagen.flow_from_directory(directory=train_path, 
                                             subset="training", 
                                             target_size=(img_width, img_height), 
                                             color_mode="rgb", 
                                             batch_size=batch_size, 
                                             class_mode="categorical", 
                                             shuffle=True, 
                                             seed=42)

val_ds = train_datagen.flow_from_directory(directory=train_path, 
                                         subset="validation", 
                                         target_size=(img_width, img_height), 
                                         color_mode="rgb", 
                                         batch_size=batch_size, 
                                         class_mode="categorical", 
                                         shuffle=False)

In [ ]:
base_model = VGGFace(model='resnet50', 
                         input_shape=(224, 224, 3), 
                         include_top=False)

base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

new_top = base_model(inputs, training=False)

new_top = layers.Flatten()(new_top)
new_top = layers.Dense(64, activation="relu")(new_top)
new_top = layers.Dropout(0.2)(new_top)

outputs = layers.Dense(2, activation='softmax')(new_top)

model = keras.Model(inputs= inputs, outputs= outputs)

In [ ]:
model.summary()

In [ ]:
INIT_LR = 1e-4
EPOCHS = 10

opt = tf.keras.optimizers.Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

mymodel = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
acc = mymodel.history['accuracy']
val_acc = mymodel.history['val_accuracy']

loss = mymodel.history['loss']
val_loss = mymodel.history['val_loss']

epochs_range = range(1, EPOCHS+1)

In [ ]:
plt.figure(figsize=(6, 6))

plt.plot(epochs_range, mymodel.history["accuracy"], label="train_acc")
plt.plot(epochs_range, mymodel.history["val_accuracy"], label="val_acc")

plt.plot(epochs_range, mymodel.history["loss"], label="train_loss")
plt.plot(epochs_range, mymodel.history["val_loss"], label="val_loss")

plt.title("Training and Validation for VGGFace2")
plt.ylim([0.0, 1.05])
plt.xticks(epochs_range)
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="right")
plt.grid()
plt.show()

In [ ]:
def classify(img_path):
    class_names = ['Correctly_Masked','Incorrectly_Masked']
    img = keras.preprocessing.image.load_img(img_path, target_size=(img_width, img_height))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = predictions[0]
    print("Class {} with {:.2f}% confidence".format(class_names[np.argmax(score)], 100*np.max(score)))

In [ ]:
# img_path = "C:\\Users\Public\Dataset\Test\WithoutMask\\85.png"
# classify(img_path)

In [ ]:
#model.save("model_vggf2_2cls.h5")

In [ ]:
from keras.models import load_model

In [ ]:
#model = load_model("model_vggf2_2cls.h5")

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

class_names = ['Correctly_Masked','Incorrectly_Masked']

x_test =[]
y_test=[]
for c in class_names:
    
    path = os.path.join('Dataset1/Test',c)
    
    label = class_names.index(c)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path, file)
        img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = keras.preprocessing.image.img_to_array(img)
        img = preprocess_input(img_array)
    

        x_test.append(img)
        y_test.append(int(label))

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
lb = LabelBinarizer()
y_test = lb.fit_transform(y_test)
y_test = to_categorical(y_test)

In [ ]:
preds = model.predict(x_test, batch_size=batch_size, verbose=1)

y_pred = np.argmax(preds, axis=1)
y_true = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_true, y_pred))

cm = confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix for VGGFace2',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(6, 4))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\nAccuracy={:0.2f}; Misclassification={:0.2f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm, [0,1], normalize=False)

In [ ]:
n_classes = y_test.shape[1]

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

precision = dict()
recall = dict()
average_precision = dict()

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], preds[:, i])
    
    average_precision[i] = average_precision_score(y_test[:, i], preds[:, i])

precision["micro"], recall["micro"], _ = precision_recall_curve(y_test.ravel(),preds.ravel())

average_precision["micro"] = average_precision_score(y_test, preds, average="micro")

In [ ]:
plt.figure()

plt.step(recall['micro'], precision['micro'], where='post', lw=2,
         label='PR curve (AP = %0.2f)' % average_precision['micro'])

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid()
plt.legend(loc='lower right')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.05])
plt.title('Precision-Recall Curve for VGGFace2')

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), preds.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()

plt.plot(fpr['micro'], tpr['micro'], color='darkorange',
         lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc['micro'])
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for VGGFace2')
plt.legend(loc="lower right")
plt.grid()
plt.show()